In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import DataFrame
from pyspark.sql import functions as F
from functools import reduce

In [ ]:
spark = SparkSession.builder.appName("App_Handler_Join").getOrCreate()